# Introdução 

Este notebook contempla o segundo projeto do curso de Data Science 1 da Udacity, e tem o objetivo didático de explorar dados de passageiros e tripulantes do Titanic com o foco na descoberta de fatos relevantes que os dados possam revelar e sua relação com a tragédia. Os dados foram baixados da plataforma da Udacity e foram retirados do [site do Kaggle](https://www.kaggle.com/c/titanic/data). 

**Autor**:  Luan Gabriel Silva Fernandes

**Data de Atualização**: 06/08/2018 

**Email**:  luangabriel70@gmail.com

**Github com este trabalho** : [repositório de lgabs](github.com/lgabs/udacity_titanic)

# 1. Visão Geral dos Dados

Importando pacotes necessários para nossa análise e lendo o conjunto de dados em `titanic-data-6.csv`:

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

df_titanic = pd.read_csv('titanic-data-6.csv')

Podemos verificar no site do Kaggle citado acima a explicação de cada atributo (coluna), reproduzida abaixo:

| Variable | Definition | Key |
| --- | --- | --- |
| survival	| Survival	| 0 = No, 1 = Yes |
|pclass	|Ticket class |	1 = 1st, 2 = 2nd, 3 = 3rd |
|sex |	Sex |	|
| Age | Age in years |	|
|sibsp | # of siblings / spouses aboard the Titanic| |	
| parch	| # of parents / children aboard the Titanic | |	
| ticket | Ticket number |	|
|fare |	Passenger fare| |	
|cabin | Cabin number | |	
| embarked | Port of Embarkation| C = Cherbourg, Q = Queenstown, S = Southampton|

Além disso, o site dá algumas notas explicativas: 

**pclass**: A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

**age**: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

**sibsp**: The dataset defines family relations in this way...

Sibling = brother, sister, stepbrother, stepsister
Spouse = husband, wife (mistresses and fiancés were ignored)

**parch**: The dataset defines family relations in this way...

Parent = mother, father
Child = daughter, son, stepdaughter, stepson

Some children travelled only with a nanny, therefore parch=0 for them.

Vamos ter uma visão geral das colunas, dos tipos de dados e das estatísticas básicas:

In [2]:
df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Nota-se a presença de algumas variáveis adicionais: 'PassengerId' (índice de referência do passageiro) e 'Name' (nome do passageiro). 

# 2. *Brainstorming* de Perguntas 

### 1. Quais fatores tem maior correlação com a propensão de algumas pessoas sobreviverem? Quais variáveis são dependentes e quais são independentes? Existem variáveis dispensáveis para a análise?
### 2. Sobrevivência, sexo e idade
   #### 2.1 Qual sexo teve maior fração de sobreviventes em relação ao total da amostra?
   #### 2.2 Observando a distribuição de idades e propondo categorias de idade (ex: jovens, adultos e idosos), podemos entender como cada uma delas sobreviveu em relação às demais?
   #### 2.3 Combinando as categorias criadas acima e as diferenças de sexo, podemos identificar qual categoria de idade e sexo mais teve "êxito" na sobrevivência? Há alguma explicação plausível para esse resultado?
### 3. Sobrevivência e classe social (avaliada pela proxy "ticket class")
   #### 3.1 Dentre os sobreviventes, qual classe social teve maior "êxito"? E qual teve o pior?
   #### 3.2 Qual a resposta da pergunta acima para os não-sobreviventes?
   #### 3.3 As respostas das duas perguntas anteriores são plausíveis, dadas as separações físicas das classes no Titanic? 
   #### 3.4 Qual sexo é "dominante" em cada classe?
   #### 3.5 Qual a faixa de preço paga por cada classe?
### 4. Sobrevivência e família
   #### 4.1 O fato de certas pessoas terem a bordo mais parentes (de qualquer tipo) influenciou na sobrevivência das mesmas?
### 5. Sobrevivência e Local de Embarque
   #### 5.1 Quantos são os sobreviventes por local de embarque? Esse fator parece ter alguma relação _de fato_ com a propensão de uma pessoa sobreviver? 
### 6. Baseando-se nas análises anteriores, quais foram os fatores que fizeram com que algumas pessoas fossem mais propensas a sobreviver?

["Enciclopédia" do Titanic: Perfil do Navio](https://www.encyclopedia-titanica.org/titanic-deckplans/profile.html)

["Enciclopédia" do Titanic: Class and Gender"](https://www.encyclopedia-titanica.org/class-gender-titanic-disaster-1912~chapter-2.html)


# 3. Limpeza dos Dados 

Vamos investigar os dados para encontrar formas de "limpá-lo":

### Valores Nulos e Atributos Relevantes

Vejamos que atributos possuem valores faltantes e as respectivas quantidades:

In [9]:
df_titanic.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Aqui notamos dados faltantes para `Age`, `Embarked` e `Cabin`. Para a nossa análise (a partir das perguntas levantadas no tópico 2), percebe-se que a informação de cabines não é relevante. Quanto à idade e cidade de embarque, são variáveis pertinentes à análise, portanto quando formos lidar com elas, teremos que tomar apenas os valores não nulos. 

Além disso, o atributo `PassengerId` não é relevante, posto que é idêntico ao próprio `index` do DataFrame. O atributo `Ticket` também não é relevante, pois as principais informações que poderíamos tirar da passagem de cada pessoa já são dadas pelo preço pago (`Fare`) e tipo de classe de viagem (`Pclass`). Por fim, `Name` também não é relevante, pois é (em tese) um valor único para cada pessoa e portanto não comparáveis (já temos outros dados que são comparáveis). Assim, vamos descartar as colunas irrelevantes para nossa análise (para evitar dúvidas, vamos primeiro verificar se não há duas pessoas com o mesmo nome):

In [25]:
df_titanic.duplicated().any()

False

In [26]:
df_titanic.drop(['PassengerId', 'Cabin', 'Ticket', 'Name'], axis=1, inplace = True)

Vejamos o resultado:

In [27]:
df_titanic.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


### Valores Duplicados

Na célula [21] notamos que não há nenhuma linha duplicada.

### Adequação de Tipos de Dados

In [28]:
df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


Pelos tipos de dados mostrados acima, todos os atributos tem tipo de dados coerente, não sendo necessário nenhuma mudança de tipos.

### Simplificando as labels das colunas 

Vamos tornar as labels minúsculas e substituir espaços por _underscores_

In [31]:
df_titanic.rename(columns = lambda s: s.strip().lower(), inplace = True)

In [32]:
df_titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [6]:
df_titanic.nunique()

PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64

# 4. Exploração dos Dados 

# 5. Conclusões  

# 6. Comunicação dos Resultados